In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date


In [2]:
def crear_features_auctions(auctions_entrenar,auctions_validar,datetime_inicio,datetime_validacion,cantidad_dias):

    auctions_entrenar['date'] = pd.to_datetime(auctions_entrenar['date'])
    auctions_validar['date'] = pd.to_datetime(auctions_validar['date'])
    
    dia_en_segundos = 86400
    auctions_entrenar['segundos'] = ((auctions_entrenar['date']-datetime_inicio).dt.seconds + 
                                     (auctions_entrenar['date']-datetime_inicio).dt.days*dia_en_segundos)

    # Cambio source_id por la probabilidad de que aparezca ese source_id
    source = auctions_entrenar['source_id'].value_counts(normalize=True)
    auctions_entrenar['source_id'] = auctions_entrenar['source_id'].replace(source)
    
    # Agrego para ref_type_id dos columnas con One Hot Encoding
    auctions_entrenar['ref_type_id'] = auctions_entrenar['ref_type_id'].astype('category')
    auctionsDummies = pd.get_dummies(auctions_entrenar['ref_type_id'], prefix = 'ref_type')
    auctions_entrenar = pd.concat([auctions_entrenar, auctionsDummies], axis=1)
    del auctions_entrenar['ref_type_id']
    
    # Agrupo el data frame por los device_id
    auctions_entrenar = auctions_entrenar.groupby('device_id').agg({'source_id':'mean','ref_type_1':'mean','ref_type_7':'mean','segundos':['min','mean','max'],'device_id':'count'})
    auctions_entrenar.columns = ['s_source_id','s_ref_type_1','s_t_min','s_t_prom','s_t_max','s_ref_type_7','s_#subastas']
    auctions_entrenar = auctions_entrenar.reset_index('device_id')
    
    auctions_validar = auctions_validar.groupby('device_id').agg({'date':'min'}).reset_index('device_id')  
    auctions_validar['segundos'] = ((auctions_validar['date']-datetime_validacion).dt.seconds + 
                                     (auctions_validar['date']-datetime_validacion).dt.days*dia_en_segundos)
    del auctions_validar['date']
        

    # Junto los data frames para tener el set de entrenamiento y el resultado esperado
    f_auctions = pd.merge(auctions_entrenar,auctions_validar,on='device_id',how='outer')
    
    # Agrego los datos faltantes
    max_tiempo = cantidad_dias*dia_en_segundos
    f_auctions['s_#subastas'] = f_auctions['s_#subastas'].fillna(0)
    f_auctions['s_source_id'] = f_auctions['s_source_id'].fillna(0)
    f_auctions['s_ref_type_1'] = f_auctions['s_ref_type_1'].fillna(0)
    f_auctions['s_ref_type_7'] = f_auctions['s_ref_type_7'].fillna(0)
    f_auctions['s_t_min'] = f_auctions['s_t_min'].fillna(max_tiempo)
    f_auctions['s_t_prom'] = f_auctions['s_t_prom'].fillna(max_tiempo)
    f_auctions['s_t_max'] = f_auctions['s_t_max'].fillna(max_tiempo)
    f_auctions['segundos'] = f_auctions['segundos'].fillna(max_tiempo)
    
    return f_auctions
    

In [3]:
auctions_entrenar = pd.read_csv('auctions_1820.csv')
auctions_validar = pd.read_csv('auctions_2123.csv')
datetime_inicio = datetime(2019,4,18)
datetime_validar = datetime(2019,4,21)
f_auctions = crear_features_auctions(auctions_entrenar,auctions_validar,datetime_inicio,datetime_validar,3)
f_auctions.to_csv('auctions_train_1820.csv', index=False)
f_auctions.head()

,device_id,s_source_id,s_ref_type_1,s_t_min,s_t_prom,s_t_max,s_ref_type_7,s_#subastas,segundos
0,41863526108385,0.122122,1.0,157228.0,185636.171429,189019.0,0.0,35.0,259200.0
1,135153013040192,0.508547,1.0,187854.0,188381.875000,188652.0,0.0,8.0,259200.0
2,161514654074162,0.249773,1.0,10366.0,10425.333333,10489.0,0.0,6.0,259200.0
3,181891380775191,0.508547,1.0,256765.0,256765.000000,256765.0,0.0,1.0,259200.0
4,186034136943920,0.508547,1.0,60166.0,160935.142857,257677.0,0.0,7.0,88.0


In [4]:
auctions_entrenar = pd.read_csv('auctions_1921.csv')
auctions_validar = pd.read_csv('auctions_2224.csv')
datetime_inicio = datetime(2019,4,19)
datetime_validar = datetime(2019,4,22)
f_auctions = crear_features_auctions(auctions_entrenar,auctions_validar,datetime_inicio,datetime_validar,3)
f_auctions.to_csv('auctions_train_1921.csv', index=False)
f_auctions.head()

,device_id,s_source_id,s_ref_type_1,s_t_min,s_t_prom,s_t_max,s_ref_type_7,s_#subastas,segundos
0,41863526108385,0.122257,1.0,70828.0,99236.171429,102619.0,0.0,35.0,259200.0
1,135153013040192,0.507354,1.0,101454.0,101981.875000,102252.0,0.0,8.0,259200.0
2,181891380775191,0.507354,1.0,170365.0,170365.000000,170365.0,0.0,1.0,259200.0
3,186034136943920,0.507354,1.0,98271.0,174949.062500,257356.0,0.0,16.0,17475.0
4,295841792051458,0.507354,1.0,173232.0,173232.000000,173232.0,0.0,1.0,259200.0


In [5]:
auctions_entrenar = pd.read_csv('auctions_2022.csv')
auctions_validar = pd.read_csv('auctions_2325.csv')
datetime_inicio = datetime(2019,4,20)
datetime_validar = datetime(2019,4,23)
f_auctions = crear_features_auctions(auctions_entrenar,auctions_validar,datetime_inicio,datetime_validar,3)
f_auctions.to_csv('auctions_train_2022.csv', index=False)
f_auctions.head()

,device_id,s_source_id,s_ref_type_1,s_t_min,s_t_prom,s_t_max,s_ref_type_7,s_#subastas,segundos
0,41863526108385,0.124705,1.0,10346.0,13671.705882,16219.0,0.0,34.0,259200.0
1,135153013040192,0.516090,1.0,15054.0,15581.875000,15852.0,0.0,8.0,259200.0
2,181891380775191,0.516090,1.0,83965.0,83965.000000,83965.0,0.0,1.0,259200.0
3,186034136943920,0.516090,1.0,11871.0,168457.019608,255971.0,0.0,51.0,3832.0
4,295841792051458,0.516090,1.0,86832.0,86832.000000,86832.0,0.0,1.0,259200.0


In [6]:
auctions_entrenar = pd.read_csv('auctions_2123.csv')
auctions_validar = pd.read_csv('auctions_2426.csv')
datetime_inicio = datetime(2019,4,21)
datetime_validar = datetime(2019,4,24)
f_auctions = crear_features_auctions(auctions_entrenar,auctions_validar,datetime_inicio,datetime_validar,3)
f_auctions.to_csv('auctions_train_2123.csv', index=False)
f_auctions.head()

,device_id,s_source_id,s_ref_type_1,s_t_min,s_t_prom,s_t_max,s_ref_type_7,s_#subastas,segundos
0,40621409780134,0.229348,1.0,226857.0,226857.000000,226857.0,0.0,1.0,259200.0
1,168103949904656,0.152259,1.0,173044.0,173044.000000,173044.0,0.0,1.0,259200.0
2,186034136943920,0.522137,1.0,88.0,122681.220339,222609.0,0.0,59.0,259200.0
3,295841792051458,0.522137,1.0,432.0,432.000000,432.0,0.0,1.0,259200.0
4,345999128501141,0.509938,1.0,164703.0,248794.166667,256365.0,0.0,24.0,250362.0
